# DATA2001 Assignment

#First, we will begin by importing the various libraries.

In [1]:
## Importing the libraries
import pandas as pd
import matplotlib as plot
import numpy as np
import geopandas as gpd

In [2]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

data_path = "."

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        HOST       = db_conn_dict['host']
        DB_USER    = db_conn_dict['user']
        DB_PW      = db_conn_dict['password']
        DEFAULT_DB = db_conn_dict['user']

        try:
            db = create_engine('postgres+psycopg2://'+DB_USER+':'+DB_PW+'@'+HOST+'/'+DEFAULT_DB, echo=False)
            conn = db.connect()
            print('connected')
        except Exception as e:
            print("unable to connect to the database")
            print(e)
        return db,conn

credfilepath = os.path.join(data_path, "data2x01_db.json")
db, conn = pgconnect(credfilepath) #connect to pgAdmin

print(db.table_names())

unable to connect to the database
(psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "soitpw11d59.shared.sydney.edu.au" (10.87.14.236) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/13/e3q8)


UnboundLocalError: local variable 'conn' referenced before assignment

The next following steps involves converting the csv files into tables on pgAdmin.

1. Uploading the statistical areas data file

2. Creating a table for the BusinessStats.csv

3. Creating a table for the Neighbourhoods.csv

Now, let's import the shape file

In [3]:
from __future__ import (absolute_import, division, print_function)
import os
import json

import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon, MultiPolygon
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras

countries = gpd.read_file( os.path.join("/Users/sidath/Desktop/data2001-assignment/1270055001_sa2_2016_aust_shape", "SA2_2016_AUST.shp") )

print(countries)

     SA2_MAIN16 SA2_5DIG16                            SA2_NAME16 SA3_CODE16  \
0     101021007      11007                             Braidwood      10102   
1     101021008      11008                               Karabar      10102   
2     101021009      11009                            Queanbeyan      10102   
3     101021010      11010                     Queanbeyan - East      10102   
4     101021011      11011                     Queanbeyan Region      10102   
...         ...        ...                                   ...        ...   
2305  901021002      91002               Cocos (Keeling) Islands      90102   
2306  901031003      91003                            Jervis Bay      90103   
2307  901041004      91004                        Norfolk Island      90104   
2308  997979799      99799  Migratory - Offshore - Shipping (OT)      99797   
2309  999999499      99499                 No usual address (OT)      99999   

                                SA3_NAME16 SA4_CODE

countries.head()

Now let's connect this to a database.

In [4]:
countries = countries.dropna()
print(countries)

     SA2_MAIN16 SA2_5DIG16               SA2_NAME16 SA3_CODE16  \
0     101021007      11007                Braidwood      10102   
1     101021008      11008                  Karabar      10102   
2     101021009      11009               Queanbeyan      10102   
3     101021010      11010        Queanbeyan - East      10102   
4     101021011      11011        Queanbeyan Region      10102   
...         ...        ...                      ...        ...   
2301  801111141      81141                  Namadgi      80111   
2304  901011001      91001         Christmas Island      90101   
2305  901021002      91002  Cocos (Keeling) Islands      90102   
2306  901031003      91003               Jervis Bay      90103   
2307  901041004      91004           Norfolk Island      90104   

                   SA3_NAME16 SA4_CODE16                    SA4_NAME16  \
0                  Queanbeyan        101                Capital Region   
1                  Queanbeyan        101                Cap

In [8]:
def pgexecute( conn, sqlcmd, args=None, msg='', silent=False ):
    """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
    retval = False
    result_set = None

    try:
        if args is None:
            result_set = conn.execute(sqlcmd).fetchall()
        else:
            result_set = conn.execute(sqlcmd, args).fetchall()

        if silent == False: 
            print("success: " + msg)
            for record in result_set:
                print(record)
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval

def pgquery( conn, sqlcmd, args=None, silent=False ):
    """ utility function to execute some SQL query statement
    can take optional arguments to fill in (dictionary)
    will print out on screen the result set of the query
    error and transaction handling built-in """
    retdf = pd.DataFrame()
    retval = False
    try:
        if args is None:
            retdf = pd.read_sql_query(sqlcmd,conn)
        else:
            retdf = pd.read_sql_query(sqlcmd,conn,params=args)
        if silent == False:
            print(retdf.shape)
            print(retdf.to_string())
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval,retdf

In [6]:
# lets just make sure we have PostGIS working on our connection
postgis_check = '''
SELECT PostGIS_Version();
'''

retval,retdf = pgquery(conn,postgis_check)
retdf

NameError: name 'conn' is not defined

In [7]:
SA2_2016 = '''CREATE TABLE sa2_2016 (
                     "SA2_MAIN16" NUMERIC, 
                     "SA2_5DIG16" NUMERIC, 
                     "SA2_NAME16" VARCHAR(50), 
                     "SA3_CODE16" NUMERIC, 
                     "SA3_NAME16" VARCHAR(50), 
                     "SA4_CODE16" NUMERIC,
                     "SA4_NAME16" VARCHAR(50), 
                     "GCC_CODE16" VARCHAR(50), 
                     "GCC_NAME16" VARCHAR(50),
                     "STE_CODE16" NUMERIC, 
                     "STE_NAME16" VARCHAR(50), 
                     "AREASQKM16" NUMERIC,
                     location GEOMETRY(MULTIPOLYGON,4326))''' 

conn.execute("DROP TABLE IF EXISTS sa2_2016")
conn.execute(SA2_2016)

NameError: name 'conn' is not defined

In [82]:
# Use GeoAlchemy's WKTElement to create a geom with SRID
# NOTE :: THIS ONLY CHANGES POLYGON's to MULTI POLYGONS, IF YOU HAVE OTHER TYPES IN YOUR DATA, 
#     YOU WILL HAVE TO CONSULT THE GEOALCHEMY AND SHAPELY DOCUMENTATION ON HOW TO HANDLE THOSE
def create_wkt_element(geom,srid):
    if (geom.geom_type == 'Polygon'):
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [83]:
srid = 4326

# since we are altering data again, we should create another copy
countriesWkCpy = countries.copy()
countriesWkCpy['location'] = countries['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
#delete the old column before insert
countriesWkCpy = countriesWkCpy.drop(columns="geometry")
countriesWkCpy

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,location
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,MULTIPOLYGON (((149.5842329970001 -35.44426999...
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,MULTIPOLYGON (((149.2189819130001 -35.36739402...
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634,MULTIPOLYGON (((149.2132479820001 -35.34324799...
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034,MULTIPOLYGON (((149.240332114 -35.347822616999...
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099,MULTIPOLYGON (((149.235800967 -35.387375302999...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,MULTIPOLYGON (((148.804064002 -35.376204003999...
2304,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,MULTIPOLYGON (((105.6739190100001 -10.41567300...
2305,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,MULTIPOLYGON (((96.83046599400006 -12.17635995...
2306,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,MULTIPOLYGON (((150.6956629970001 -35.18296700...


In [84]:
# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
SA2_2016_table_name = "sa2_2016"
countriesWkCpy.to_sql(SA2_2016_table_name, conn, if_exists='append', index=False, 
                         dtype={'location': Geometry('MULTIPOLYGON', srid)})

In [85]:
query = "SELECT COUNT(*) FROM sa2_2016"
retval,retdf = pgquery(conn, query)
retdf

(1, 1)
   count
0   2292


,count
0,2292
